In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('../input/titanic/train.csv')
data.head()

In [ ]:
y = data['Survived']
sex = pd.get_dummies(data['Sex'],drop_first=True)
x=data.drop(['PassengerId','Survived','Name','Sex','Ticket','Cabin','Embarked'],axis=1)
x=pd.concat([x,sex],axis=1)

x.head()


In [ ]:
DATA = pd.concat([x,y],axis = 1)
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
g=sns.heatmap(DATA.corr(),annot=True,cmap="RdYlGn")

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test = train_test_split(x,y, random_state = 0, test_size = 0.2)
print(x_test)

In [ ]:
from xgboost import XGBClassifier

params={
 "n_estimators"     : [10,100,200,500,800,1000,1200,1500,1700,2000,2200,2400,2600,2800,3000],
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7, 8,9,10 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4, 0.5,0.6 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}
model = XGBClassifier()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
random_search=RandomizedSearchCV(model,param_distributions=params,n_iter=30,n_jobs=-1,cv=5,verbose=3)


In [ ]:
random_search.fit(x_train,y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:

model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=4,
              min_child_weight=3, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = model.predict(x_test)
print(predictions)


In [ ]:
accuracy = accuracy_score(predictions, y_test)
print("the mean absolute error is ", accuracy)

In [ ]:
test=pd.read_csv('../input/titanic/test.csv')
testD = test
test.head()

In [ ]:
sexTest = pd.get_dummies(test['Sex'],drop_first=True)
test=test.drop(['PassengerId','Name','Sex','Ticket','Cabin','Embarked'],axis=1)
test=pd.concat([test,sexTest],axis=1)
test.head()



In [ ]:
predictionTest = model.predict(test)
print(predictionTest)

In [ ]:
submission = pd.DataFrame({
    "PassengerId": testD['PassengerId'],
    "Survived": predictionTest
})

submission.to_csv('submission.csv', index = False)

In [ ]:
submission = pd.read_csv('submission.csv')
print(submission)